# Facial Recognition: Transfer Learning Video Capture

### Libraries Used:

In [1]:
# Libraries used for OpenCv and path creation/reading
import cv2
import os
from os import listdir

# Keras libraries necessary for the model creation
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation
from keras.layers import Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

# Keras library for VGG16 model
from keras.applications.vgg16 import VGG16

# Libraries for preprocessing the data
from PIL import Image
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array, save_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.utils import to_categorical
from glob import glob

# Importing numpy to work with arrays, matplotlib for visuals
# and pandas for dataframe.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy import spatial

In [2]:
def cosineSimilarity(dataset_img, captured_img):
    cos_dis = spatial.distance.cosine(dataset_img, captured_img)
    cos_sim = 1 - cos_dis
    return cos_sim

In [3]:
tl_model = load_model('frv_model.h5')

In [4]:
user_image_path = r'C:/Users/jpasz/Facial Recognition Images'
user_images = dict()
for i in listdir(user_image_path):
    if i != '.DS_Store':  # for mac users
        current_folder = user_image_path + '/' + i
        image_predictions = []
        for j in listdir(current_folder):
            current_image = current_folder+'/'+j
            current_user_face = load_img(current_image, target_size=(224, 224))
            face_array = img_to_array(current_user_face)
            face_expanded = np.expand_dims(face_array, axis=0)
            face_preprocessed = preprocess_input(face_expanded)
            image_predictions.append(tl_model.predict(face_preprocessed)[0,:])
        user_images[i] = image_predictions
        

In [7]:

camera = cv2.VideoCapture(0)
while(True):
    result, image = camera.read()
    if result:
        user_detected = False
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
        detected_faces = face_cascade.detectMultiScale(image, 1.1, 4)
        for (x, y, w, h) in detected_faces:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            face_recognized = image[y:y + h, x:x + w]
            face_recognized = cv2.resize(face_recognized, dsize=(224, 224))
            face_array = img_to_array(face_recognized)
            face_expanded = np.expand_dims(face_array, axis=0)
            face_preprocessed = preprocess_input(face_expanded)
            video_prediction = tl_model.predict(face_preprocessed)[0, :]
            for i in user_images:
                user_folder = user_images[i]
                for j in user_folder:
                    cosine_similarity = cosineSimilarity(j, video_prediction)
                    if(cosine_similarity <= 0.25):
                        cv2.putText(image, i, (int(x+w+20), int(y-15)), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 0), 1)
                        user_detected=True
                        break
 
            if(user_detected==False):
                cv2.putText(image, 'Unrecogized User', (int(x+w+20), int(y-15)), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 0), 3)
     
    cv2.imshow('Facial Recognition', image)
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break
camera.release()
cv2.destroyAllWindows()           
            
            
        
